In [ ]:
# Install the requirements
pip install -r requirements.txt

In [21]:
import openai
from langchain_openai import AzureChatOpenAI
from langchain.sql_database import SQLDatabase
from langchain.agents import create_sql_agent
from sqlalchemy import create_engine
import urllib


In [22]:
# Set up your OpenAI API key
# api_key = 'fe1a8d2af7bb4ec984fbe47a13081bcb' # swec
# api_endpoint = 'https://peterlil-openai.openai.azure.com/'
api_key = '80e61c36e95a4fe298c80b2a65a518af' # eus
api_endpoint = 'https://peterlil-openai-eus.openai.azure.com/'
open_ai_api_version = '2024-06-01'

In [23]:
# Define your Azure SQL Database connection string
params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=tcp:peterlilopenaisqlsrv.database.windows.net,1433;"
    "DATABASE=testdb;"
    "UID=peterlil;"
    "PWD=@1hkKBEYlQhaFGrV;"
    "Encrypt=yes;"
    "TrustServerCertificate=no;"
)

# Create a SQLAlchemy engine
sql_engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")


In [24]:

# Initialize the AzureOpenAI object with Azure-specific parameters
llm = AzureChatOpenAI(model='gpt-4o',
                      deployment_name='gpt-4o',
                      azure_endpoint=api_endpoint,
                      api_key=api_key,
                      api_version=open_ai_api_version,
                      temperature=0)

In [25]:
# print(sql_engine.url)
connection_string = str(sql_engine.url)

# Create a new SQLDatabase instance using the connection string
sql_database = SQLDatabase.from_uri(connection_string)
 


mssql+pyodbc:///?odbc_connect=DRIVER%3D%7BODBC+Driver+17+for+SQL+Server%7D%3BSERVER%3Dtcp%3Apeterlilopenaisqlsrv.database.windows.net%2C1433%3BDATABASE%3Dtestdb%3BUID%3Dpeterlil%3BPWD%3D%401hkKBEYlQhaFGrV%3BEncrypt%3Dyes%3BTrustServerCertificate%3Dno%3B


In [26]:
# Create the SQL agent
agent = create_sql_agent(llm=llm, db=sql_database)

In [35]:
# Define your natural language query
# natural_language_query = "List all the tables in the schema SalesLT."
natural_language_query = "List the first and last names of 10 customers."
# natural_language_query = "Show me the last 10 orders."
# natural_language_query = "Show me all customers who placed a sales order in the last recorded month."

In [36]:
# Generate the SQL query
sql_query = agent.invoke(natural_language_query)
print(sql_query)

{'input': 'List the first and last names of 10 customers.', 'output': 'The first and last names of 10 customers are:\n1. Orlando Gee\n2. Keith Harris\n3. Donna Carreras\n4. Janet Gates\n5. Lucy Harrington\n6. Rosmarie Carroll\n7. Dominic Gash\n8. Kathleen Garza\n9. Katherine Harding\n10. Johnny Caprio'}
